#### 참고 Python GDAL/OGR Cookbook
https://pcjericks.github.io/py-gdalogr-cookbook/index.html

In [1]:
import gdal
import glob
from osgeo import ogr, osr, gdal
import sys
import numpy as np

우선 메타데이터 읽어보자
idx, inv, ctl 파일이 있다

In [10]:
with open('gfsanl_3_20040401_0000_000.txt','r') as fp:
    inv = fp.read()
    bands = inv.split('\n')

In [11]:
for band in bands[:]: #len(bands)==284
    print(band)

1:0:D=2004040100:HGT:1000 mb:kpds=7,100,1000:anl:winds are N/S:"Geopotential height [gpm]
2:105968:D=2004040100:HGT:975 mb:kpds=7,100,975:anl:winds are N/S:"Geopotential height [gpm]
3:211936:D=2004040100:HGT:950 mb:kpds=7,100,950:anl:winds are N/S:"Geopotential height [gpm]
4:317904:D=2004040100:HGT:925 mb:kpds=7,100,925:anl:winds are N/S:"Geopotential height [gpm]
5:423872:D=2004040100:HGT:900 mb:kpds=7,100,900:anl:winds are N/S:"Geopotential height [gpm]
6:529840:D=2004040100:HGT:850 mb:kpds=7,100,850:anl:winds are N/S:"Geopotential height [gpm]
7:643954:D=2004040100:HGT:800 mb:kpds=7,100,800:anl:winds are N/S:"Geopotential height [gpm]
8:758068:D=2004040100:HGT:750 mb:kpds=7,100,750:anl:winds are N/S:"Geopotential height [gpm]
9:872182:D=2004040100:HGT:700 mb:kpds=7,100,700:anl:winds are N/S:"Geopotential height [gpm]
10:986296:D=2004040100:HGT:650 mb:kpds=7,100,650:anl:winds are N/S:"Geopotential height [gpm]
11:1100410:D=2004040100:HGT:600 mb:kpds=7,100,600:anl:winds are N/S:"Geo

In [12]:
with open('gfsanl_3_20040401_0000_000_ctl.txt','r') as fp:
    ctl = fp.read()
print(ctl)

DSET ^gfsanl_3_20040401_0000_000.grb

INDEX ^gfsanl_3_20040401_0000_000.idx
undef 9.999E+20
*
title Global Forecast System : 20040401 0000 UTC cycle - Analysis only
*
options yrev
dtype grib 3
xdef 360 linear   0 1
ydef 181 linear -90 1
tdef 1 linear 00:00z01apr2004 3hr
zdef 26 levels
1000 975 950 925 900 850 800 750 700 650 600 550 500 450 400 350 300 
250 200 150 100 70 50 30 20 10 
*
vars 145
no4LFTX  0 132,1,0      surface Best (4-layer) lifted index [K]
no5WAVH  0 222,100,500  ** -5-wave geopotential height [gpm]
ABSV     26 41,100,0    Absolute vorticity [/s]
ACPCP    0 63,1,0       surface Convective precipitation [kg/m^2] 
APCP     0 61,1,0       surface Total precipitation [kg/m^2] 
CAPE     0 157,1,0      surface Convective Avail. Pot. Energy [J/kg]
CAPE180_0mb 0 157,116,46080 ** 180-0 mb above gnd Convective Avail. Pot. Energy [J/kg]
oCFRZR   0 141,1,0      surface Categorical freezing rain [yes=1 no=0] 
oCICEP   0 142,1,0      surface Categorical ice pellets [yes=1 no=0] 
C

데이터인 grb를 읽어보자

In [2]:
ds =gdal.Open('gfsanl_3_20040401_0000_000.grb') # grb를 읽어 gdal.Dataset으로 받아옴

In [10]:
type(ds)

osgeo.gdal.Dataset

In [15]:
# try:
#     from osgeo import ogr, osr, gdal
# except:
#     sys.exit('ERROR: cannot find GDAL/OGR modules')

# # example GDAL error handler function
# def gdal_error_handler(err_class, err_num, err_msg):
#     errtype = {
#             gdal.CE_None:'None',
#             gdal.CE_Debug:'Debug',
#             gdal.CE_Warning:'Warning',
#             gdal.CE_Failure:'Failure',
#             gdal.CE_Fatal:'Fatal'
#     }
#     err_msg = err_msg.replace('\n',' ')
#     err_class = errtype.get(err_class, 'None')
#     print('Error Number: %s' % (err_num))
#     print('Error Type: %s' % (err_class))
#     print('Error Message: %s' % (err_msg))

# if __name__=='__main__':

#     # install error handler
#     gdal.PushErrorHandler(gdal_error_handler)

#     # Raise a dummy error
#     gdal.Error(1, 2, 'test error')

#     #uninstall error handler
#     gdal.PopErrorHandler()

Error Number: 2
Error Type: Debug
Error Message: test error


- 본격적으로 데이터를 읽어보자
    - 래스터 데이터에 대한 개략적인 설명 : https://docs.qgis.org/2.18/ko/docs/gentle_gis_introduction/raster_data.html
    - 래스터 데이터는 X, Y 좌표에 band라는 레이어를 쌓는 방식으로 구성됨

In [6]:
#래스터 데이터의 band값을 읽어오자
# this allows GDAL to throw Python Exceptions
gdal.UseExceptions()

try:
    src_ds =gdal.Open('gfsanl_3_20040401_0000_000.grb')
except RuntimeError as e:
    print('Unable to open')
    print(e)
    sys.exit(1)

try:
    srcband = src_ds.GetRasterBand(1)
except RuntimeError as  e:
    # for example, try GetRasterBand(10)
    print('Band ( %i ) not found' % band_num)
    print(e)
    sys.exit(1)

In [5]:
srcband.GetStatistics( True, True ) #Minimum, Maximum, Mean, StdDev를 읽어오는 메소드

[-408.6, 267.6, 88.205992063492, 117.18251968807]

In [8]:
# 밴드의 카운트만큼 각 밴드의 기초통계값을 찍어준다
if src_ds is None:
    print('Unable to open')
    sys.exit(1)

print("[ RASTER BAND COUNT ]: ", src_ds.RasterCount)
for band in range( src_ds.RasterCount ):
    band += 1
    print("[ GETTING BAND ]: ", band)
    srcband = src_ds.GetRasterBand(band)
    if srcband is None:
        continue

    stats = srcband.GetStatistics( True, True )
    if stats is None:
        continue

    print("[ STATS ] =  Minimum=%.3f, Maximum=%.3f, Mean=%.3f, StdDev=%.3f" % ( \
                stats[0], stats[1], stats[2], stats[3] ))

[ RASTER BAND COUNT ]:  284
[ GETTING BAND ]:  1
[ STATS ] =  Minimum=-408.600, Maximum=267.600, Mean=88.206, StdDev=117.183
[ GETTING BAND ]:  2
[ STATS ] =  Minimum=-209.100, Maximum=468.200, Mean=295.158, StdDev=118.924
[ GETTING BAND ]:  3
[ STATS ] =  Minimum=-5.400, Maximum=684.800, Mean=506.814, StdDev=122.267
[ GETTING BAND ]:  4
[ STATS ] =  Minimum=202.600, Maximum=905.900, Mean=723.393, StdDev=127.184
[ GETTING BAND ]:  5
[ STATS ] =  Minimum=415.300, Maximum=1131.900, Mean=945.109, StdDev=133.430
[ GETTING BAND ]:  6
[ STATS ] =  Minimum=854.400, Maximum=1599.800, Mean=1405.123, StdDev=149.319
[ GETTING BAND ]:  7
[ STATS ] =  Minimum=1311.800, Maximum=2104.900, Mean=1889.629, StdDev=169.165
[ GETTING BAND ]:  8
[ STATS ] =  Minimum=1786.700, Maximum=2638.400, Mean=2401.375, StdDev=192.227
[ GETTING BAND ]:  9
[ STATS ] =  Minimum=2285.400, Maximum=3210.300, Mean=2943.253, StdDev=217.918
[ GETTING BAND ]:  10
[ STATS ] =  Minimum=2815.500, Maximum=3819.000, Mean=3519.219, S

#### 래스터 데이터를 np.array로 다뤄보자

In [11]:
print(ds.RasterXSize, ds.RasterYSize, ds.RasterCount) #dataset의 사이즈

360 181 284


In [3]:
dsA = ds.ReadAsArray() # array로 읽어온다

In [14]:
dsA.shape # 3차원구조

(284, 181, 360)

In [4]:
dsT = dsA.transpose() # x,y 좌표별로 데이터를 다루고 싶기에 일단 transpose 시켜본다

In [16]:
dsT.shape

(360, 181, 284)

In [17]:
dsT[0][0].shape #그 결과 인덱싱하면 band별 관측치를 보여준다

(284,)

In [18]:
cells = []
for x in range(ds.RasterXSize):
    for y in range(ds.RasterYSize):
        cells.append( (x,y, *dsT[x][y]) ) # x,y좌표를 기준으로 band별 관측치를 보려고 한다
    

In [23]:
A = np.array(cells) #cells 리스트를 다시 array로 변환한다
A.shape #  결과는 360*181 == 65160의 row와 2+ 284의 col을 가진 이차원 array, 2004.04.01 00시의 데이터이다

(65160, 286)

#### 주성분 분석으로 feature의 수를 줄여보자

In [20]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
pca = PCA(n_components=50) # n_components 몇으로 해야할까? 일단 50으로 해보자

In [32]:
X = A[:,2:] #좌표정보를 제외한 데이터

In [33]:
pca.fit(X) #fitting

PCA(copy=True, iterated_power='auto', n_components=50, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [34]:
Z = pca.transform(X)

In [35]:
Z # 50차원으로 feature 수가 줄어들었다 이런식으로 줄어든 차원의 데이터를 클러스터링에 쓸수있다

array([[ 1.00589995e+21,  3.19612828e+18, -2.04071607e+04, ...,
         1.30535187e+04,  3.81542022e+03,  1.09076842e+04],
       [ 1.00589995e+21,  3.19612828e+18, -2.01824403e+04, ...,
         1.30581766e+04,  3.81647057e+03,  1.09083671e+04],
       [ 1.00589995e+21,  3.19612828e+18, -1.95624982e+04, ...,
         1.30733719e+04,  3.82412183e+03,  1.08998697e+04],
       ...,
       [-9.93900048e+20,  3.15706195e+18,  5.69318949e+04, ...,
        -1.29499209e+04, -3.96441137e+03, -1.08298403e+04],
       [-9.93900048e+20,  3.15706195e+18,  5.71224897e+04, ...,
        -1.29533622e+04, -3.96040497e+03, -1.08310124e+04],
       [-9.93900048e+20,  3.15706195e+18,  5.75329009e+04, ...,
        -1.29571404e+04, -3.96437898e+03, -1.08352400e+04]])

#### PCA를 위해 생각해볼 것

1. PCA가 필요한가? 직접 필요한 변수 골라낼수없을까
2. 날짜별로 feature의 수가 다른데 일괄적으로 차원축소 가능할까

이후 구현 필요

In [141]:
# w, U = np.linalg.eig(pca.get_covariance())

In [43]:
# U.T.dot(A.T).T

In [44]:
# plt.scatter(Z[:,0], Z[:,1], c='r', s=100)
# plt.title("transformed data")
# plt.show()